In [28]:
import pandas as pd
import numpy as np
import nltk
import os.path
from nltk.stem import PorterStemmer
import RAKE
import operator
import spacy
import collections

In [47]:
data_path = 'data/reviewSelected100.json'
os.path.exists(data_path)

True

In [48]:
data = pd.read_json(data_path, lines=True,encoding = "ISO-8859-1")
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11


In [9]:
# Select business b1 and prepare B1
uniqueBusiness = data['business_id'].unique()
b1 = uniqueBusiness[0]
B1 = data.loc[data['business_id'] == b1]
B1 = pd.DataFrame(B1, columns = ['text'])
B1.rename(columns={"text": "unstemmed_text"}, inplace = True)

In [13]:
# Stemming
ps = PorterStemmer()
B1['stemmed_text'] = B1['unstemmed_text'].apply(lambda x: ps.stem(x))

In [16]:
print(type(B1))

<class 'pandas.core.frame.DataFrame'>


In [14]:
print(B1['stemmed_text'])

0       we had my mother's birthday party here on 10/2...
7       nice atmosphere, plenty of tv's. priced afford...
21      the location at wt harris and statesville road...
77      i went to this location to eat one evening.  t...
157     this edition of excited about eating takes us ...
                              ...                        
1712    placed an order through postmates and left a n...
1723    our experience tonight was well above expectat...
1751    brunch menu && $4 mimosas/bloody mary's on sun...
1776    best burgers and fries!! ate there twice while...
1786    pretty great place for a set down lunch. they ...
Name: stemmed_text, Length: 100, dtype: object


In [15]:
print(B1['unstemmed_text'])

0       We had my Mother's Birthday Party here on 10/2...
7       Nice atmosphere, plenty of TV's. Priced afford...
21      The location at WT Harris and Statesville Road...
77      I went to this location to eat one evening.  T...
157     This edition of Excited about Eating takes us ...
                              ...                        
1712    Placed an order through postmates and left a n...
1723    Our experience tonight was well above expectat...
1751    Brunch menu && $4 mimosas/bloody mary's on Sun...
1776    Best burgers and fries!! Ate there twice while...
1786    Pretty great place for a set down lunch. They ...
Name: unstemmed_text, Length: 100, dtype: object


In [22]:
stop_dir = "data/SmartStoplist.txt"
os.path.exists(data_path)


True

In [24]:
#rake_object = RAKE.Rake(stop_dir,test = False)

TypeError: __init__() got an unexpected keyword argument 'test'

In [26]:
nlp = spacy.load('en_core_web_sm')
noun_adj_pairs = []


for i in B1['stemmed_text'] :
  doc = nlp(i)
  for i,token in enumerate(doc):
      if token.pos_ not in ('NOUN','PROPN'):
          continue
      for j in range(i+1,len(doc)):
          if doc[j].pos_ == 'ADJ':
              noun_adj_pairs.append((str(token),str(doc[j])))
              break
noun_adj_pairs

[('mother', 'great'),
 ('birthday', 'great'),
 ('party', 'great'),
 ('10/29/16', 'great'),
 ('time', 'great'),
 ('food', 'great'),
 ('music', 'great'),
 ('waiters', 'great'),
 ('nice', 'good'),
 ('atmosphere', 'good'),
 ('plenty', 'good'),
 ('tv', 'good'),
 ('.', 'good'),
 ('food', 'good'),
 ('dinner', 'good'),
 ('dinner', 'good'),
 ('menu', 'good'),
 ('bbq', 'good'),
 ('chicken', 'good'),
 ('nachos', 'good'),
 ('location', 'least'),
 ('wt', 'least'),
 ('harris', 'least'),
 ('statesville', 'least'),
 ('road', 'least'),
 ('cafeteria', 'least'),
 ('office', 'least'),
 ('staff', 'least'),
 ('week', 'good'),
 ('lunch', 'good'),
 ('today', 'good'),
 ('fact', 'good'),
 ('beer', 'good'),
 ('list', 'good'),
 ('food', 'good'),
 ('service', 'handy'),
 ('street', 'handy'),
 ('office', 'handy'),
 ('lunch', 'great'),
 ('work', 'great'),
 ('oyster', 'great'),
 ('night', 'great'),
 ('thursdays', 'great'),
 ('deal', 'great'),
 ('pint', 'great'),
 ('night', 'great'),
 ('tuesdays', 'great'),
 ('location

In [39]:
#Find the total number of adjective phrases
print(len(noun_adj_pairs))

2182


In [29]:
#Find the top 10 adjective phrases
counter=collections.Counter(noun_adj_pairs)
print(counter.most_common(10))

[(('food', 'good'), 16), (('food', 'great'), 8), (('staff', 'friendly'), 7), (('place', 'great'), 7), (('table', 'general'), 5), (('group', 'little'), 4), (('bar', 'good'), 4), (('beer', 'cold'), 4), (('place', 'good'), 4), (('waiters', 'great'), 3)]


In [35]:
B1_stemmed_text = B1['stemmed_text'].to_frame()

In [49]:
# Exploring the entire dataset's adjective phrases
data = pd.DataFrame(data, columns = ['text'])
data.rename(columns={"text": "unstemmed_text"}, inplace = True)

In [50]:
print(data['unstemmed_text'])

0        We had my Mother's Birthday Party here on 10/2...
1        Good Korean grill near Eaton Centre. The marin...
2        Was recommended to try this place by few peopl...
3        Ambience: Would not expect something this nice...
4        Absolutely the WORST pool company that I have ...
                               ...                        
15295    This was the worst experience ever. So much so...
15296    We come here every time we hit Vegas! A giant ...
15297    As locals we used  to the this place when it w...
15298    The food was delicious. We were seated in 15 m...
15299    Wonderful spot (the patio) and OH SO EXCELLENT...
Name: unstemmed_text, Length: 15300, dtype: object


In [54]:
# Exploring the entire dataset's adjective phrases
data = pd.DataFrame(data, columns = ['text'])
data.rename(columns={"text": "unstemmed_text"}, inplace = True)


# Stemming
ps = PorterStemmer()
data['stemmed_text'] = data['unstemmed_text'].astype(str).apply(lambda x: ps.stem(x))

In [59]:
nlp = spacy.load('en_core_web_sm')
noun_adj_pairs_2 = []


for i in data['stemmed_text']  :
  doc = nlp(i)
  for i,token in enumerate(doc):
      if token.pos_ not in ('NOUN','PROPN'):
          continue
      for j in range(i+1,len(doc)):
          if doc[j].pos_ == 'ADJ':
              noun_adj_pairs_2.append((str(token),str(doc[j])))
              break
noun_adj_pairs_2

[]